In [ ]:
import cv2
import os
import numpy as np
import tensorflow as tf

from random import shuffle
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import Attention

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)#learning rate scheduling

inputs = Input(shape=[10, IMG_SIZE, IMG_SIZE, 3], name="input")
convrnn = Conv2D(64, (3, 3), padding='same')(inputs)
convrnn = BatchNormalization()(convrnn)
convrnn = Activation('relu')(convrnn)
convrnn = Conv2D(64, (3, 3))(convrnn)
convrnn = BatchNormalization()(convrnn)
convrnn = Activation('relu')(convrnn)
convrnn = MaxPooling2D((2, 2))(convrnn)
convrnn = Conv2D(64, (3, 3))(convrnn)
convrnn = BatchNormalization()(convrnn)
convrnn = Activation('relu')(convrnn)
convrnn = MaxPooling2D((2, 2))(convrnn)
convrnn = Conv2D(64, (3, 3))(convrnn)
convrnn = BatchNormalization()(convrnn)
convrnn = Activation('relu')(convrnn)
convrnn = MaxPooling2D((2, 2))(convrnn)
convrnn = Flatten()(convrnn)

convrnn = tf.reshape(convrnn, (1, 10, 6400))

lstm_fw = LSTM(units=32, return_sequences=True)
lstm_bw = LSTM(units=32, go_backwards=True, return_sequences=True)
bi_lstm = Bidirectional(lstm_fw, backward_layer=lstm_bw)(convrnn)


attention = Attention()([bi_lstm, bi_lstm])#attention mechanism

convrnn = Dense(64)(attention)
convrnn = BatchNormalization()(convrnn)
convrnn = Activation('relu')(convrnn)
convrnn = Dense(32)(convrnn)
convrnn = BatchNormalization()(convrnn)
convrnn = Activation('relu')(convrnn)
output = Dense(1, activation='sigmoid')(convrnn)

model = tf.keras.Model(inputs=inputs, outputs=output)

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_val, y_val), callbacks=[reduce_lr])
